# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Experiment, Workspace
from azureml.core.compute import AmlCompute, ComputeTarget 

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,choice
from azureml.core import ScriptRunConfig
import os

## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()
experiment_name = 'Medical-Insurance-Premium-Prediction'

experiment=Experiment(ws, experiment_name)

run = experiment.start_logging()

In [3]:
try:
    compute_cluster = ComputeTarget(ws,'ml-clstr')
except:
    compute_cluster_conf = AmlCompute.provisioning_configuration(
        vm_size="Standard_DS12_V2",
        vm_priority = 'lowpriority',
        min_nodes=0,
        max_nodes=4,
        idle_seconds_before_scaledown=240)
    compute_cluster = ComputeTarget.create(ws,"ml-clstr",compute_cluster_conf)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=4,slack_factor=0.2,)

#TODO: Create the different params that you will be using during training
param_sampling = RandomParameterSampling({
    '--n_estimators':choice(25,50,75,100,125,150),
    '--max_depth':choice(2,4,6,8,10,12,14,16,18,20),
})

#TODO: Create your estimator and hyperdrive config
estimator = SKLearn(
    source_directory=".",
    compute_target=compute_cluster,
    entry_script="train.py"
)

hyperdrive_run_config = HyperDriveConfig(
    hyperparameter_sampling=param_sampling,
    primary_metric_name="RMSE",
    primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
    max_total_runs=30,
    max_concurrent_runs=4,
    policy=early_termination_policy,
    estimator=estimator
    )

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [5]:
#TODO: Submit your experiment

cap_hd_submit = experiment.submit(config=hyperdrive_run_config)

In [6]:
RunDetails(cap_hd_submit).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

In [7]:
cap_hd_submit.wait_for_completion(show_output=True)

RunId: HD_0c4e3c3e-def8-4c9e-800d-b7f710bb47de
Web View: https://ml.azure.com/runs/HD_0c4e3c3e-def8-4c9e-800d-b7f710bb47de?wsid=/subscriptions/e8e8da26-0fa0-48b5-a094-ad8115fd47b3/resourcegroups/udacityamlnd/workspaces/workspaceone&tid=8a794c71-bef8-4d45-8cb2-b0c18444be8b

Streaming azureml-logs/hyperdrive.txt

"<START>[2022-01-02T12:47:00.466116][API][INFO]Experiment created<END>\n""<START>[2022-01-02T12:47:01.310482][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-02T12:47:01.813198][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n""<START>[2022-01-02T12:47:31.353584][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2022-01-02T12:47:31.567231][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"<START>[2022-01-02T12:48:00.8692089Z][SCHEDULER][INFO]Scheduling job, id='HD_0c4e3c3e-def8-4c9

{'runId': 'HD_0c4e3c3e-def8-4c9e-800d-b7f710bb47de',
 'target': 'ml-clstr',
 'status': 'Completed',
 'startTimeUtc': '2022-01-02T12:47:00.180819Z',
 'endTimeUtc': '2022-01-02T13:07:38.369094Z',
 'services': {},
 'properties': {'primary_metric_config': '{"name": "RMSE", "goal": "minimize"}',
  'resume_from': 'null',
  'runTemplate': 'HyperDrive',
  'azureml.runsource': 'hyperdrive',
  'platform': 'AML',
  'ContentSnapshotId': 'd0520af0-a520-4af9-8f88-9bc0defda47f',
  'user_agent': 'python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0',
  'space_size': '60',
  'score': '4485.884647338268',
  'best_child_run_id': 'HD_0c4e3c3e-def8-4c9e-800d-b7f710bb47de_27',
  'best_metric_status': 'Succeeded'},
 'inputDatasets': [],
 'outputDatasets': [],
 'logFiles': {'azureml-logs/hyperdrive.txt': 'https://workspaceone4929537462.blob.core.windows.net/azureml/ExperimentRun/dcid.HD_0c4e3c3e-def8-4c9e-800d-b7f710bb47de/azurem

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [6]:
for key, val in cap_hd_submit.get_properties().items():
    print(f"{key}: \n{val}\n")

primary_metric_config: 
{"name": "RMSE", "goal": "minimize"}

resume_from: 
null

runTemplate: 
HyperDrive

azureml.runsource: 
hyperdrive

platform: 
AML

ContentSnapshotId: 
714bb27c-6145-413c-ad4f-ecbac4378a22

user_agent: 
python/3.6.9 (Linux-5.4.0-1063-azure-x86_64-with-debian-buster-sid) msrest/0.6.21 Hyperdrive.Service/1.0.0 Hyperdrive.SDK/core.1.36.0

space_size: 
60

score: 
4495.279797438918

best_child_run_id: 
HD_f17a2163-141b-46b8-b691-339b1f166df8_3

best_metric_status: 
Succeeded



In [7]:
cap_best_run_hd = cap_hd_submit.get_best_run_by_primary_metric()

In [8]:
#TODO: Save the best model
cap_best_model_hd = cap_best_run_hd.register_model(model_name = 'MIPP-HD',model_path = '.')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

